In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': 'C:\\AgodaGit\\passivbot\\' + filename}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_95460/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_95460/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_95460/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_95460/1315120660.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_inde

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%
0,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.909573,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.114573,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.838780,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.952475,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.242079,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.841806,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
730,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.4,ZECUSDT,0.966600,True,True,0.0600,0.0179,100.0,100.0,0.9792,12.53,3.573
731,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.4,ZECUSDT,0.966600,True,True,0.0600,0.0179,100.0,100.0,0.9792,12.53,3.573
732,C:\AgodaGit\passivbot\backtests/binance\ZECUSD...,5.5,ZECUSDT,0.966000,True,True,0.0599,0.0173,100.0,100.0,0.9792,12.52,3.435


Now filter the dataframe

In [4]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["WAVESUSDT",
"SUSHIUSDT",
"RLCUSDT",
"BNBUSDT",]
big_data['adgt_%'] = big_data['adgl_%']+big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              & (big_data['adgt_%'] > 0.1 )
              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,ebrl,long,short,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrs,tgl_%,tgs_%,adgt_%
577,C:\AgodaGit\passivbot\backtests/binance\RLCUSD...,5.5,RLCUSDT,0.9551,True,True,0.0522,0.0606,100.0,100.0,0.9691,10.660,12.66,0.1128
578,C:\AgodaGit\passivbot\backtests/binance\RLCUSD...,5.5,RLCUSDT,0.9357,True,True,0.1000,0.1330,100.0,100.0,0.9042,23.230,29.72,0.2330
628,C:\AgodaGit\passivbot\backtests/binance\SUSHIU...,5.5,SUSHIUSDT,0.8284,True,True,0.0436,0.0750,100.0,100.0,0.9176,20.020,14.92,0.1186
629,C:\AgodaGit\passivbot\backtests/binance\SUSHIU...,5.5,SUSHIUSDT,0.8284,True,True,0.0436,0.0750,100.0,100.0,0.9176,20.020,14.92,0.1186
630,C:\AgodaGit\passivbot\backtests/binance\SUSHIU...,5.5,SUSHIUSDT,0.9103,True,True,0.0174,0.1160,100.0,100.0,0.9027,12.540,24.64,0.1334
675,C:\AgodaGit\passivbot\backtests/binance\WAVESU...,5.5,WAVESUSDT,0.9477,True,True,0.0652,0.0703,100.0,100.0,0.9267,13.750,15.09,0.1355
676,C:\AgodaGit\passivbot\backtests/binance\WAVESU...,5.5,WAVESUSDT,0.9151,True,True,0.0268,0.0874,100.0,100.0,0.8851,7.495,18.16,0.1142


In [5]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_95460/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
577,C:\AgodaGit\passivbot\backtests/binance\RLCUSD...,RLCUSDT,0.1128,0.0522,0.0606,100.0,100.0,0.9551,0.9691
578,C:\AgodaGit\passivbot\backtests/binance\RLCUSD...,RLCUSDT,0.2330,0.1000,0.1330,100.0,100.0,0.9357,0.9042
628,C:\AgodaGit\passivbot\backtests/binance\SUSHIU...,SUSHIUSDT,0.1186,0.0436,0.0750,100.0,100.0,0.8284,0.9176
630,C:\AgodaGit\passivbot\backtests/binance\SUSHIU...,SUSHIUSDT,0.1334,0.0174,0.1160,100.0,100.0,0.9103,0.9027
675,C:\AgodaGit\passivbot\backtests/binance\WAVESU...,WAVESUSDT,0.1355,0.0652,0.0703,100.0,100.0,0.9477,0.9267
676,C:\AgodaGit\passivbot\backtests/binance\WAVESU...,WAVESUSDT,0.1142,0.0268,0.0874,100.0,100.0,0.9151,0.8851
